In [1]:
import json
import time
import torch
import transformers
import wandb
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from transformers import BitsAndBytesConfig


# nltk 다운로드 (최초 1회)
nltk.download('punkt')

# wandb 초기화
wandb.init(project="evaluations", name="8B_origin")

# corpus.json 파일 로드 (UTF-8 인코딩)
with open("./corpus.json", "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

# 10개만 추론
evaluation_data = evaluation_data[:10]

# 허깅페이스 방식으로 모델 및 파이프라인 초기화
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
generator = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)
generator.model.eval()

# 시스템 프롬프트 정의
SYSTEM_PROMPT = (
    "You are a helpful AI assistant. Please answer the user's questions kindly. "
    "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."
)

# 채팅 템플릿을 적용하여 프롬프트를 생성하는 함수
def make_prompt(system_msg: str, user_msg: str) -> str:

    msg = f"""
다음은 유튜브 타임라인 요약 문서입니다. 이 내용을 분석하여 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.
중요하지 않더라도 알면 좋을 추가 핵심 용어도 더해주세요.

[타임라인 요약]:
{user_msg}

[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는 여러 개의 키포인트 리스트로 구성됩니다.
- 반드시 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

"""

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": msg}
    ]
    prompt = generator.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return prompt

# 종료 토큰 설정
terminators = [
    generator.tokenizer.eos_token_id,
    generator.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]



# 평가 루프: corpus.json에 있는 각 예제에 대해 추론 수행
for idx, example in enumerate(evaluation_data):
    print(f"\n==== Example {idx+1} ====")

    # 사용자 입력과 프롬프트 생성
    user_input = example["input"]
    prompt = make_prompt(SYSTEM_PROMPT, user_input)

    # 입력 토큰 인코딩
    encoded_input = generator.tokenizer(prompt, return_tensors="pt")
    encoded_input = {k: v.to(generator.model.device) for k, v in encoded_input.items()}
    prompt_length = encoded_input["input_ids"].shape[1]

    # 속도 및 메모리 측정을 위한 시작 시간 기록 및 GPU 메모리 초기화
    start_time = time.time()
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(generator.model.device)

    # 모델 생성: 토큰별 score 반환 포함
    outputs = generator.model.generate(
        **encoded_input,
        max_new_tokens=256,
        eos_token_id=terminators[0],
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # GPU 동기화 후 종료 시간 기록
    if torch.cuda.is_available():
        torch.cuda.synchronize(generator.model.device)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # GPU 메모리 사용량 측정 (Peak 메모리, MB 단위)
    if torch.cuda.is_available():
        peak_memory = round(torch.cuda.max_memory_allocated(generator.model.device) / (1024 ** 3), 1)
    else:
        peak_memory = 0

    # 생성된 전체 토큰에서 프롬프트 이후의 candidate 토큰 추출
    generated_ids = outputs.sequences[0]
    candidate_ids = generated_ids[prompt_length:]
    candidate_text = generator.tokenizer.decode(candidate_ids, skip_special_tokens=True).strip()

    print("Candidate output:")
    print(candidate_text)
    print(f"Inference Time: {elapsed_time:.2f} seconds")
    print(f"Peak GPU Memory Usage: {peak_memory:.2f} GB")

    # 참조 텍스트: evaluation_data의 "output"을 JSON 문자열로 변환
    reference_text = json.dumps(example["output"], ensure_ascii=False, indent=2)

    # BLEU 점수 계산 (공백 기준 토큰화)
    candidate_tokens = candidate_text.split()
    reference_tokens = reference_text.split()
    bleu = sentence_bleu(
        [reference_tokens],
        candidate_tokens,
        smoothing_function=SmoothingFunction().method1
    )
    print(f"BLEU score: {bleu:.4f}")

    # 로그 확률 계산: 각 생성 토큰에 대해 log‑softmax 후 선택된 토큰의 로그 확률 합산
    log_score = 0.0
    for i, step_scores in enumerate(outputs.scores):
        log_probs = torch.log_softmax(step_scores, dim=-1)
        token_id = candidate_ids[i].unsqueeze(0)  # shape: (1,)
        token_log_prob = log_probs[0, token_id].item()
        log_score += token_log_prob
    print(f"Log probability score (sum): {log_score:.4f}")

    # wandb에 메트릭 로깅
    wandb.log({
        "example_index": idx,
        "BLEU_score": bleu,
        "Inference_time_sec": elapsed_time,
        "Peak_GPU_Memory_GB": peak_memory,
        "Log_probability_score": log_score
    })

wandb.finish()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: Currently logged in as: iamkimhongil92 (iamkimhongil92-lumenasoft) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



==== Example 1 ====


From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Candidate output:
[키 포인트]
```
[
  {
    "term": "캐릭터 해금 시스템",
    "description": "게임에서 플레이 지속성을 높이기 위해 플레이어가 캐릭터를 해금하는 기능을 구현하는 시스템."
  },
  {
    "term": "버튼 추가",
    "description": "게임 내에서 캐릭터 해금을 위한 추가 버튼을 삽입하는 과정."
  },
  {
    "term": "Ctrl+D",
    "description": "윈도우에서 복사 기능을 수행하는 단축키."
  },
  {
    "term": "감자 농부, 콩 농부",
    "description": "게임 내에서 해금할 수 있는 두 가지 캐릭터 이름."
  },
  {
    "term": "잠금 상태 버튼",
    "description": "기본 캐릭터를 해금 상태로 두고, 나머지 캐릭터에 대한 잠금 상태를 설정하는 버튼."
  },
  {
    "term": "속성 수정",
    "description": "잠금 상태 버튼의 색깔과 아이콘을 설정하고 관리하기 쉽게 구조를 잡는 과정."
  },
  {
    "term": "업적 달성",
    "description": "플레이어가 특정 조건을 충족하면 해금할 수 있는 캐릭터를 언락하는 시스템."
  },
  {
    "term": "스크립트",
    "description": "프로그래밍 언어를 사용하여 게임 로직을 구현하는 코드."
  },
  {
    "term": "언락 조건",
    "description": "해금할 수 있는 캐릭터의 언락 조건을 정의하는 과정."
  },
  {
    "term": "UI 변화",
    "description": "업적 달성 시 버튼에 대한 변화를 나타내는 사용자 인터페이스 요소."
  },
  {
    "term": "데이터 저장",
    "description": "게임이 종료된 후에도 해금 상태를 유지하기 위해 플레이

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Candidate output:
```json
[
  {
    "term": "오디오 시스템",
    "description": "게임 내 오디오를 관리하고 재생하는 유니티의 기본 기능 및 구성 요소"
  },
  {
    "term": "사운드",
    "description": "게임의 사실감을 추가하는 중요한 요소로, 배경음악과 효과음으로 구성"
  },
  {
    "term": "오디오 매니저",
    "description": "유니티에서 오디오 소스를 관리하고 재생하는 오브젝트"
  },
  {
    "term": "오디오 클립",
    "description": "오디오 파일이 아닌 오디오 클립을 사용하여 오디오 소스를 재생하는 방법"
  },
  {
    "term": "오디오 소스",
    "description": "오디오 매니저에서 재생되는 오디오 데이터"
  },
  {
    "term": "채널",
    "description": "여러 개의 효과음을 동시에 재생하기 위해 사용되는 오디오 소스의 그룹"
  },
  {
    "term": "오디오 소스 변수",
    "description": "효과음의 상태를 관리하기 위해 사용되는 변수"
  },
  {
    "term": "배경음악",
    "description": "게임 배경에 지속적으로 재생되는 음악"
  },
  {
    "term": "효과음",
    "description": "게임 이벤트나 액션에 맞춰 재생되는 짧은 오디오 파일"
  },
  {
    "term": "사운드 품질",
    "description": "오디오 소스의 음질을 조절하는 설정"
  },
  {
    "term": "테스트",
    "description": "완성된 오디오 매니저와 소스를 실제 게임 환경에서 확인하고 최종 점검"
  },
  {
    "term": "음향적인 문제",
    "description": "게임 환경에서 발생할 수 있는 오디오

BLEU_score,▁▂▂▃▁▅▁▁▅▁▁▂▅▂▁▅▂█▆▂
Inference_time_sec,▅▃▄▅▂▆▅█▃▂▁▇▄▄▃▃▄▁▁▂
Log_probability_score,▄▆▄▅▇▄▄▁▅▅▆▃▃▄▄█▅▇▇▄
Peak_GPU_Memory_GB,▁▅▄▆▄▅▇█▆▃▂▅▆▅▄▄▅▃▁▂
example_index,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
BLEU_score,0.02367
Inference_time_sec,13.53784
Log_probability_score,-151.90753
Peak_GPU_Memory_GB,15.8
example_index,19
